In [1]:
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
from tensorflow.keras.utils import to_categorical
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
from sklearn.metrics import classification_report, accuracy_score, roc_curve, auc, recall_score
from sklearn.model_selection import train_test_split, TimeSeriesSplit
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
import sqlite3
import pandas as pd
from keras.callbacks import EarlyStopping

In [3]:
conn = sqlite3.connect(r'C:\Users\spiri\codingbootcamp\Final-project-C4Y\data.db')
cursor = conn.cursor()

In [4]:
query = ('''

Select * 
from inv_campaign_eval
left join client
on inv_campaign_eval.client_id = client.client_id
left join client_products
on inv_campaign_eval.client_id = client_products.client_id
left join balances
on inv_campaign_eval.client_id = balances.client_id''')
df = pd.read_sql_query(query, conn)

df.head()

,client_id,poutcome,client_id,age,job,marital,education,gender,client_id,has_deposits,loan,has_insurance,has_mortgage,date,balance,client_id,currency
0,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,2022-06-21 16:21:15.291346,2633.671253,249789938,CZK
1,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,2022-07-05 16:21:15.291346,2663.697443,249789938,CZK
2,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,2022-07-19 16:21:15.291346,2605.350108,249789938,CZK
3,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,2022-08-02 16:21:15.291346,2694.840314,249789938,CZK
4,249789938,success,249789938,38.0,services,married,secondary,M,249789938,yes,no,n,no,2022-08-16 16:21:15.291346,2656.018182,249789938,CZK


In [6]:
df = df.T.drop_duplicates().T
df.job.fillna(value='unknown', inplace=True)
df.age.fillna(value=df.age.mean(), inplace=True)
df.balance = pd.to_numeric(df.balance)
df.client_id = pd.to_numeric(df.client_id)

C:\Users\spiri\AppData\Local\Temp\ipykernel_3152\2789236670.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df.job.fillna(value='unknown', inplace=True)
C:\Users\spiri\AppData\Local\Temp\ipykernel_3152\2789236670.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, 

In [8]:
rate = {'CZK': 1, 'USD': 23, 'EUR': 25}

df.balance = df.apply(lambda row: row['balance'] * rate[row['currency']], axis=1)


In [11]:
df.dtypes

client_id                 int64
poutcome                 object
age                     float64
job                      object
marital                  object
education                object
gender                   object
has_deposits             object
loan                     object
has_insurance            object
has_mortgage             object
date             datetime64[ns]
balance                 float64
currency                 object
dtype: object

In [10]:
df.date = pd.to_datetime(df.date)
df.age = pd.to_numeric(df.age)

In [12]:
import pandas as pd
from sklearn.linear_model import LinearRegression

# Assuming you have a DataFrame df with columns 'client_id', 'date', and 'deposit_value'
# Replace df with your actual DataFrame containing the deposit data

# Group data by client_id
grouped = df.groupby('client_id')

# Initialize a dictionary to store results
trend_results = {}

# Iterate over groups (clients)
for client_id, group_data in grouped:
    # Extract deposit values and corresponding dates
    dates = group_data['date'].values.reshape(-1, 1)
    deposits = group_data['balance'].values
    
    # Fit linear regression model
    model = LinearRegression()
    model.fit(dates, deposits)
    
    # Get slope (trend) for the client
    slope = model.coef_[0]
    
    # Store result
    trend_results[client_id] = slope

# Print or store trend results for each client
for client_id, slope in trend_results.items():
    print(f"Client {client_id}: Trend (slope) = {slope}")

Client 100488560: Trend (slope) = 9.399688678462699e-16
Client 100605013: Trend (slope) = -2.461530214141e-16
Client 102604801: Trend (slope) = -1.6312570863116585e-15
Client 104030718: Trend (slope) = 1.8260437058860294e-15
Client 106830158: Trend (slope) = -1.686037842752745e-15
Client 107099542: Trend (slope) = -1.1561933387924304e-16
Client 107813918: Trend (slope) = 2.8877574408164413e-16
Client 108338626: Trend (slope) = 5.697353506532008e-16
Client 109346132: Trend (slope) = 7.041704572031028e-16
Client 110980600: Trend (slope) = -9.345175500255335e-17
Client 111146953: Trend (slope) = 1.960829557230758e-15
Client 112377679: Trend (slope) = 2.306780962209291e-15
Client 112580373: Trend (slope) = 2.342086960882894e-15
Client 112861558: Trend (slope) = 6.482238773138055e-16
Client 112913283: Trend (slope) = -8.759041982404582e-16
Client 113164248: Trend (slope) = -2.5327061856915483e-17
Client 114920096: Trend (slope) = -5.709131166059662e-16
Client 115014363: Trend (slope) = 4.94